## Naturkartan
* this [Notebook](https://github.com/salgo60/svenska-vandringsleder/blob/main/Notebook/Naturkartan.ipynb)   

API
* [API](https://api.naturkartan.se/v2/sites/search.geojson)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-09-20 17:45:13.042474


## API

In [4]:
urlNaturkartan = "https://api.naturkartan.se/v2/sites/search.geojson"
header ="accept: text/plain;charset=utf-8"
import urllib3, json
import pandas as pd 

http = urllib3.PoolManager()
r = http.request(
    'GET',
    urlNaturkartan,
    headers={
        "accept": "application/json"
    })

print(r.status)
data = json.loads(r.data.decode('utf-8')) 
print(type(data))
#print(r.info()) 

200
<class 'dict'>


In [33]:

from pandas.io.json import json_normalize
Naturkartan = json_normalize(data["features"])

<ipython-input-33-824419bf9d55>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  Naturkartan = json_normalize(data["features"])


In [37]:
Naturkartan

,id,type,geometry.type,geometry.coordinates,properties.url,properties.link,properties.name,properties.slug,properties.tags,properties.time,...,properties.wheelchair_tested,properties.business_site_data,properties.show_in_brand_site,properties.trail_status_reported_at,properties.business_site_data.address,properties.business_site_data.telephone,properties.business_site_data.website_url,properties.business_site_data.opening_hours,properties.color,properties.parent
0,18546,Feature,Point,"[16.4748505691457, 66.3160608416262]",https://www.naturkartan.se/sv/norrbottenslan/l...,None,"Dass Kattugglekojan, Laisdalen",laisdalen-fur-dass-vid-kattugglekojan,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18547,Feature,Point,"[16.4894806445951, 66.2808717169018]",https://www.naturkartan.se/sv/norrbottenslan/l...,None,"Dass Bárasjuhka, Laisdalen",laisdalen-fur-dass-barrastugan,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18523,Feature,Point,"[16.9704617765231, 66.2410323852801]",https://www.naturkartan.se/sv/norrbottenslan/l...,None,"Informationsskylt Hällbacken, Laisdalen",laisdalen-fur-informationstavla-hallbacken,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16724,Feature,Point,"[17.3718579395161, 60.1902146008368]",https://www.naturkartan.se/sv/upplandsstiftels...,None,"Utsiktsplats, Iggelbo",utsiktsplats-iggelbo,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8631,Feature,Point,"[17.4701607227325, 59.7352970701782]",https://www.naturkartan.se/sv/uppsalalan/rastp...,None,"Rastplats, Wikparken",rastplats-wikparken-3,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,12953,Feature,Point,"[18.6791918052287, 68.3102143133751]",https://www.naturkartan.se/sv/norrbottenslan/t...,None,Dass Abiskojåkka,tc-torrdass-4,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
727,1879,Feature,Point,"[18.001083, 59.439732]",https://www.naturkartan.se/sv/taby/hundbad-2,http://www.taby.se/hunditaby,Hundbadet vid Rösjön,hundbad-2,[],NaN,...,False,NaN,True,NaN,,NaN,None,[],NaN,NaN
728,6198,Feature,Point,"[18.0608364179077, 59.2123892104539]",https://www.naturkartan.se/sv/huddinge/agestab...,None,Fornborgen vid Ågesta,agestaborgen,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
729,16055,Feature,Point,"[11.9337059341437, 57.5045442752106]",https://www.naturkartan.se/sv/halland/parkerin...,None,Parkering Särö Västerskog,parkering-saro-vasterskog,[],NaN,...,False,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
Naturkartan.to_csv("Baturkartan.csv")
Naturkartan.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 57 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   id                                           731 non-null    object 
 1   type                                         731 non-null    object 
 2   geometry.type                                731 non-null    object 
 3   geometry.coordinates                         731 non-null    object 
 4   properties.url                               649 non-null    object 
 5   properties.link                              199 non-null    object 
 6   properties.name                              650 non-null    object 
 7   properties.slug                              650 non-null    object 
 8   properties.tags                              650 non-null    object 
 9   properties.time                              0 non-null      float64
 10  pr

In [ ]:
# properties.url 

## Wikidata

SPARQL

In [29]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

queryNaturkartan = """ SELECT ?item ?itemLabel ?WDvalue
    {
      ?item wdt:P10467 ?WDvalue 
            SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
    }
"""



def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDNaturkartan = get_sparql_dataframe(endpoint_url, queryNaturkartan)



In [30]:
WDNaturkartan.sample(4)

,item,itemLabel,WDvalue
502,http://www.wikidata.org/entity/Q107212024,Lindholmens utegym,stockholms-lan/lindholmens-utegym
206,http://www.wikidata.org/entity/Q106709723,"Kärrhuset, badplats",ange/ojesjon-karrhuset-badplats
42,http://www.wikidata.org/entity/Q106711877,Åsmansbo badplats,smedjebacken/asmansbo-bad
386,http://www.wikidata.org/entity/Q108155374,Backetjärn badplats,ange/backetjarn-badplats


In [31]:
WDNaturkartan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item       818 non-null    object
 1   itemLabel  818 non-null    object
 2   WDvalue    818 non-null    object
dtypes: object(3)
memory usage: 19.3+ KB


In [39]:
Naturkartan["properties.url"]

0      https://www.naturkartan.se/sv/norrbottenslan/l...
1      https://www.naturkartan.se/sv/norrbottenslan/l...
2      https://www.naturkartan.se/sv/norrbottenslan/l...
3      https://www.naturkartan.se/sv/upplandsstiftels...
4      https://www.naturkartan.se/sv/uppsalalan/rastp...
                             ...                        
726    https://www.naturkartan.se/sv/norrbottenslan/t...
727         https://www.naturkartan.se/sv/taby/hundbad-2
728    https://www.naturkartan.se/sv/huddinge/agestab...
729    https://www.naturkartan.se/sv/halland/parkerin...
730    https://www.naturkartan.se/sv/huddinge/entre-p...
Name: properties.url, Length: 731, dtype: object

In [43]:
Naturkartan["WDvalue"] = Naturkartan["properties.url"].str.replace('https://www.naturkartan.se/sv/','')

<ipython-input-43-4b2d79bf7610>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Naturkartan["WDvalue"] = Naturkartan["properties.url"].str.replace('https://www.naturkartan.se/sv/','')


In [46]:
# merge properties.url WDvalue 

mergeNaturkartan = pd.merge(WDNaturkartan, Naturkartan,how='outer', on='WDvalue',indicator=True)   
mergeNaturkartan.rename(columns={"_merge": "WD_Naturkartan"},inplace = True)
#mergeSKBLWD['WD_Naturkartan'] = mergeSKBLWD['WD_SKBL'].str.replace('left_only','WD_only').str.replace('right_only','SKBL_only')
#mergeSKBLWD["WD_SKBL"].value_counts()  

In [50]:
mergeNaturkartan["WD_Naturkartan"].value_counts()

left_only     784
right_only    699
both           34
Name: WD_Naturkartan, dtype: int64

In [53]:
mergeNaturkartan[{"item","itemLabel","WDvalue","properties.url","WD_Naturkartan"}]

<ipython-input-53-56752ae52e2c>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  mergeNaturkartan[{"item","itemLabel","WDvalue","properties.url","WD_Naturkartan"}]


,itemLabel,properties.url,WDvalue,WD_Naturkartan,item
0,Bergs kommun,NaN,berg,left_only,http://www.wikidata.org/entity/Q845930
1,Alvesta kommun,NaN,alvesta,left_only,http://www.wikidata.org/entity/Q182007
2,Kristianstads kommun,NaN,kristianstad,left_only,http://www.wikidata.org/entity/Q498857
3,Överkalix kommun,NaN,overkalix,left_only,http://www.wikidata.org/entity/Q307569
4,Askersunds kommun,NaN,askersund,left_only,http://www.wikidata.org/entity/Q509501
...,...,...,...,...,...
1512,NaN,https://www.naturkartan.se/sv/halland/gula-sli...,halland/gula-slingan,right_only,NaN
1513,NaN,https://www.naturkartan.se/sv/norrbottenslan/t...,norrbottenslan/tc-torrdass-4,right_only,NaN
1514,NaN,https://www.naturkartan.se/sv/huddinge/agestab...,huddinge/agestaborgen,right_only,NaN
1515,NaN,https://www.naturkartan.se/sv/halland/parkerin...,halland/parkering-saro-vasterskog,right_only,NaN


In [54]:
Naturkartan_only = mergeNaturkartan[mergeNaturkartan["WD_Naturkartan"]=="right_only"]  

In [62]:
pd.set_option('max_colwidth', 600)
pd.set_option('display.max_rows', None)

Naturkartan_only["properties.url"]

818                   https://www.naturkartan.se/sv/norrbottenslan/laisdalen-fur-dass-vid-kattugglekojan
819                          https://www.naturkartan.se/sv/norrbottenslan/laisdalen-fur-dass-barrastugan
820              https://www.naturkartan.se/sv/norrbottenslan/laisdalen-fur-informationstavla-hallbacken
821                                https://www.naturkartan.se/sv/upplandsstiftelsen/utsiktsplats-iggelbo
822                                       https://www.naturkartan.se/sv/uppsalalan/rastplats-wikparken-3
823                                https://www.naturkartan.se/sv/uppsalalan/parkeringsplats-graso-gard-3
824                       https://www.naturkartan.se/sv/stockholmsstad/informationsskylt-vid-kanaanbadet
825                         https://www.naturkartan.se/sv/stockholmsstad/informationsskylt-vid-ridskolan
826                                  https://www.naturkartan.se/sv/uppsalalan/busshallplats-graso-gard-2
827                              https://www.naturkarta